In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the Lice`nse is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI SDK：批量预测自动ML训练图像对象检测模型

<table align="left">
 
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/automl/sdk_automl_image_object_detection_batch.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/automl/sdk_automl_image_object_detection_batch.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/automl/sdk_automl_image_object_detection_batch.ipynb" target='_blank'>      
    <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI工作台中打开
    </a>
  </td>                    
</table>
<br/><br/><br/>

## 概述

本教程演示如何使用Vertex AI SDK创建图像目标检测模型，并使用Google Cloud的AutoML模型进行批量预测。

了解更多关于图像数据的目标检测，请访问[Object detection for image data](https://cloud.google.com/vertex-ai/docs/training-overview#object_detection_for_images)。

### 目标

在本教程中，您将从Python脚本中创建一个AutoML图像对象检测模型，然后使用Vertex AI SDK进行批量预测。您也可以使用`gcloud`命令行工具或在Cloud Console上在线创建和部署模型。

本教程使用以下Google Cloud ML服务：

- `AutoML训练`
- `Vertex AI数据集`

执行的步骤包括：

- 创建一个Vertex `数据集`资源。
- 训练模型。
- 查看模型评估。
- 进行批量预测。

在使用批量预测和在线预测之间有一个关键区别：

* 预测服务：对整个实例集（即一个或多个数据项）进行即时预测，并实时返回结果。

* 批量预测服务：在后台对整个实例集进行排队（批量）预测，并在准备就绪时将结果存储在Cloud Storage存储桶中。

### 数据集

本教程使用的数据集是来自[TensorFlow数据集](https://www.tensorflow.org/datasets/catalog/open_images_v4)中的沙拉类别的[OpenImages数据集](https://www.tensorflow.org/datasets/catalog/open_images_v4)。这个数据集不需要任何特征工程。您将在本教程中使用的数据集版本存储在公共云存储桶中。训练好的模型可以预测图像中五种项目类别之一的沙拉、海鲜、番茄、烘焙食品或奶酪的边界框位置和相应类型。

### 费用

本教程使用 Google Cloud 的计费组件：

* Vertex AI
* 云存储

了解 [Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing) 和 [云存储价格](https://cloud.google.com/storage/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/) 来根据您的预期使用量生成一个费用估算。

## 安装

安装最新版本的Python云存储、Bigquery和Vertex AI SDK。

In [ ]:
# Install the packages.
! pip3 install --upgrade google-cloud-aiplatform \
                         google-cloud-storage     \
                         tensorflow -q

只有协作：取消注释以下单元格以重新启动内核

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

在你开始之前

### 设置您的项目ID

**如果您不知道您的项目ID**，请尝试以下操作：
* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 参阅支持页面：[查找项目ID](https://support.google.com/googleapi/answer/7014113)。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

#### 区域

您还可以更改 Vertex AI 使用的 `REGION` 变量。了解有关 [Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations) 的更多信息。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

### 验证您的 Google 云账户

根据您的 Jupyter 环境，您可能需要手动进行验证。请按照下面的相关说明进行操作。

1. 顶点 AI 工作台
* 无需操作，因为您已经完成了身份验证。

2. 本地JupyterLab实例，取消注释并运行：

In [ ]:
# ! gcloud auth login

3. 合作，取消注释并运行:

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

4. 服务账号或其他
* 参考https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples了解如何向您的服务账号授予云存储权限。

创建一个云存储桶

创建一个存储桶，用于存储中间产物，如数据集。

In [ ]:
BUCKET_URI = "gs://your-bucket-name-unique"  # @param {type:"string"}

只有在您的存储桶尚不存在时：运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

### 导入库

In [ ]:
import google.cloud.aiplatform as aiplatform

使用以下Python代码来初始化Vertex AI SDK，为您的项目和相应的存储桶准备好环境。

In [ ]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_URI, location=REGION)

# 教程

现在您已准备好开始创建您自己的AutoML图像目标检测模型。

云存储培训数据的位置。

现在将变量 `IMPORT_FILE` 设置为 Cloud Storage 中 CSV 索引文件的位置。

In [ ]:
IMPORT_FILE = "gs://cloud-samples-data/vision/salads.csv"

快速查看您的数据

本教程使用存储在公共云存储桶中的Salads数据集的版本，使用CSV索引文件。

首先快速查看数据。通过计算CSV索引文件中的行数（`wc -l`）来计算示例数，然后查看前几行数据。

In [ ]:
count = ! gsutil cat $IMPORT_FILE | wc -l
print("Number of Examples", int(count[0]))

print("First 10 rows")
! gsutil cat $IMPORT_FILE | head -10

### 创建数据集

接下来，使用`ImageDataset`类的`create`方法创建`Dataset`资源，该方法需要以下参数：

- `display_name`：`Dataset`资源的可读名称。
- `gcs_source`：要将数据项导入`Dataset`资源的一个或多个数据集索引文件列表。
- `import_schema_uri`：数据项的数据标记模式。

此操作可能需要几分钟时间。

In [ ]:
DISPLAY_NAME = "salads_unique"
dataset = aiplatform.ImageDataset.create(
    display_name=DISPLAY_NAME,
    gcs_source=[IMPORT_FILE],
    import_schema_uri=aiplatform.schema.dataset.ioformat.image.bounding_box,
)

print(dataset.resource_name)

### 创建并运行训练管道

要训练一个AutoML模型，需要执行两个步骤：1）创建训练管道，2）运行管道。

#### 创建训练管道

使用`AutoMLImageTrainingJob`类创建一个AutoML训练管道，具有以下参数：

- `display_name`：`TrainingJob`资源的人类可读名称。
- `prediction_type`：训练模型的任务类型。
  - `classification`：图像分类模型。
  - `object_detection`：图像目标检测模型。
- `multi_label`：如果是分类任务，是否为单标签（`False`）或多标签（`True`）。
- `model_type`：部署的模型类型。
  - `CLOUD`：部署在谷歌云上。
  - `CLOUD_HIGH_ACCURACY_1`：针对谷歌云部署时的精度优先于延迟的优化。
  - `CLOUD_LOW_LATENCY_`：针对谷歌云部署时的延迟优先于精度的优化。
  - `MOBILE_TF_VERSATILE_1`：部署在边缘设备上。
  - `MOBILE_TF_HIGH_ACCURACY_1`：针对边缘设备部署时的精度优先于延迟的优化。
  - `MOBILE_TF_LOW_LATENCY_1`：针对边缘设备部署时的延迟优先于精度的优化。
- `base_model`：（可选）从现有的`Model`资源进行迁移学习--仅支持图像分类。

实例化的对象是训练作业的任务。

In [ ]:
job = aiplatform.AutoMLImageTrainingJob(
    display_name=DISPLAY_NAME,
    prediction_type="object_detection",
    multi_label=False,
    model_type="CLOUD",
    base_model=None,
)

print(job)

#### 运行训练管道

接下来，您可以通过调用方法`run`，使用以下参数来运行作业以启动训练工作：

- `dataset`：用于训练模型的`Dataset`资源。
- `model_display_name`：经过训练的模型的可读名称。
- `training_fraction_split`：用于训练的数据集百分比。
- `test_fraction_split`：用于测试（保留数据）的数据集百分比。
- `validation_fraction_split`：用于验证的数据集百分比。
- `budget_milli_node_hours`：（可选）以毫小时为单位指定的最大训练时间（1000 = 小时）。
- `disable_early_stopping`：如果为`True`，则服务认为无法进一步提高模型目标测量时，训练可能会在使用完整预算之前完成。

当`run`方法完成时，将返回`Model`资源。

训练管道的执行时间最长为1小时30分钟。

In [ ]:
model = job.run(
    dataset=dataset,
    model_display_name=DISPLAY_NAME,
    training_fraction_split=0.8,
    validation_fraction_split=0.1,
    test_fraction_split=0.1,
    budget_milli_node_hours=20000,
    disable_early_stopping=False,
)

## 查看模型评估分数
在模型训练完成后，您可以查看其评估分数。

首先，您需要获取对新模型的引用。与数据集一样，您可以使用部署模型时创建的模型变量的引用，或者列出项目中的所有模型。

In [ ]:
# Get model resource ID
filter_name = f"display_name={DISPLAY_NAME}"
models = aiplatform.Model.list(filter=filter_name)

# Get a reference to the Model Service client
client_options = {"api_endpoint": f"{REGION}-aiplatform.googleapis.com"}
model_service_client = aiplatform.gapic.ModelServiceClient(
    client_options=client_options
)

model_evaluations = model_service_client.list_model_evaluations(
    parent=models[0].resource_name
)
model_evaluation = list(model_evaluations)[0]
print(model_evaluation)

发送批量预测请求

向您部署的模型发送批量预测。

获取测试项
现在对您的 Vertex 模型进行批量预测。您将使用数据集中的任意示例作为测试项。不要担心这些示例可能已被用于训练模型 — 我们只是想演示如何进行预测。

In [ ]:
test_items = !gsutil cat $IMPORT_FILE | head -n2
cols_1 = str(test_items[0]).split(",")
cols_2 = str(test_items[1]).split(",")
if len(cols_1) == 11:
    test_item_1 = str(cols_1[1])
    test_label_1 = str(cols_1[2])
    test_item_2 = str(cols_2[1])
    test_label_2 = str(cols_2[2])
else:
    test_item_1 = str(cols_1[0])
    test_label_1 = str(cols_1[1])
    test_item_2 = str(cols_2[0])
    test_label_2 = str(cols_2[1])

print(test_item_1, test_label_1)
print(test_item_2, test_label_2)

### 复制测试项

对于批量预测，请将测试项目复制到您的云存储存储桶中。

In [ ]:
file_1 = test_item_1.split("/")[-1]
file_2 = test_item_2.split("/")[-1]

! gsutil cp $test_item_1 $BUCKET_URI/$file_1
! gsutil cp $test_item_2 $BUCKET_URI/$file_2

test_item_1 = BUCKET_URI + "/" + file_1
test_item_2 = BUCKET_URI + "/" + file_2

制作批量输入文件

现在制作一个批量输入文件，您将把它存储在您的本地云存储桶中。批量输入文件可以是CSV或JSONL格式。在本教程中您将使用JSONL格式。对于JSONL文件，您需要为每个数据项（实例）的每一行制作一个字典条目。该字典包含以下键/值对：

- `content`：图像的云存储路径。
- `mime_type`：内容类型。在我们的示例中，是一个“jpeg”文件。

例如：

                        {'content': '[您的存储桶]/file1.jpg', 'mime_type': 'jpeg'}

In [ ]:
import json
import os

import tensorflow as tf

gcs_input_uri = BUCKET_URI + "/test.jsonl"
with tf.io.gfile.GFile(gcs_input_uri, "w") as f:
    data = {"content": test_item_1, "mime_type": "image/jpeg"}
    f.write(json.dumps(data) + "\n")
    data = {"content": test_item_2, "mime_type": "image/jpeg"}
    f.write(json.dumps(data) + "\n")

print(gcs_input_uri)
! gsutil cat $gcs_input_uri

### 发起批量预测请求

现在您的模型资源已经训练好，您可以通过调用batch_predict()方法进行批量预测，使用以下参数：

- `job_display_name`：批量预测作业的可读名称。
- `gcs_source`：一个或多个批量请求输入文件的列表。
- `gcs_destination_prefix`：用于存储批量预测结果的Cloud Storage位置。
- `machine_type`：在专用资源上运行批量预测的机器类型。如果不指定机器类型，将使用自动资源运行批量预测作业。
- `starting_replica_count`：批量操作开始时使用的机器副本数量。如果未设置，则Vertex AI决定起始数量，不超过`max_replica_count`。仅在设置了`machine_type`时使用。
- `max_replica_count`：批量操作可能扩展到的最大机器副本数量。仅在设置了`machine_type`时使用。默认值为10。
- `sync`：如果设置为True，则调用将阻塞，等待异步批处理作业完成。

对于自动机器学习模型，只支持手动扩展。在手动扩展中，starting_replica_count和max_replica_count的值相同。
对于此批处理作业，我们正在使用手动扩展。在这里，我们将starting_replica_count和max_replica_count都设置为相同的值1。

In [ ]:
batch_predict_job = model.batch_predict(
    job_display_name=DISPLAY_NAME,
    gcs_source=gcs_input_uri,
    gcs_destination_prefix=BUCKET_URI,
    machine_type="n1-standard-4",
    starting_replica_count=1,
    max_replica_count=1,
    sync=False,
)

print(batch_predict_job)

等待批量预测作业完成

接下来，等待批量作业完成。或者，可以在`batch_predict()`方法中将参数`sync`设置为`True`，以阻止，直到批量预测作业完成。

In [ ]:
batch_predict_job.wait()

### 获取预测结果

接下来，从已完成的批量预测作业中获取结果。

结果将被写入您在批量预测请求中指定的云存储输出桶中。您可以调用iter_outputs()方法来获取生成的结果的每个云存储文件的列表。每个文件以JSON格式包含一个或多个预测请求：

- `content`：预测请求。
- `prediction`：预测响应。
 - `ids`：每个预测请求的内部分配的唯一标识符。
 - `displayNames`：每个类别标签的类名。
 - `bboxes`：每个检测到对象的边界框。

In [ ]:
import json

bp_iter_outputs = batch_predict_job.iter_outputs()

prediction_results = list()
for blob in bp_iter_outputs:
    if blob.name.split("/")[-1].startswith("prediction"):
        prediction_results.append(blob.name)

tags = list()
for prediction_result in prediction_results:
    gfile_name = f"gs://{bp_iter_outputs.bucket.name}/{prediction_result}"
    with tf.io.gfile.GFile(name=gfile_name, mode="r") as gfile:
        for line in gfile.readlines():
            line = json.loads(line)
            print(line)

清理

要清理此项目中使用的所有Google Cloud资源，您可以删除用于本教程的[Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源：

- 数据集
- 模型
- AutoML训练作业
- 批处理作业
- 云存储桶

In [ ]:
delete_bucket = False

# Delete the dataset using the Vertex dataset object
dataset.delete()

# Delete the model using the Vertex model object
model.delete()

# Delete the AutoML or Pipeline trainig job
job.delete()

# Delete the batch prediction job using the Vertex batch prediction object
batch_predict_job.delete()

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -r $BUCKET_URI